# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a drugstore. Specifically, this report will be targeted to stakeholders interested in opening a **Japanese restaurant** in **Kuala Lumpur, Malaysia**.

Since there are lots of restaurants i in Kuala Lumpur, we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Japanese restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each are will then be clearly experssed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Japanese restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of location, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centres of those areas will be obtained using **GeoPy reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* number of retail stores and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Kuala Lumpur centre will be obtained using **GeoPy geocoding** of well known Kuala Lumpur location (KLCC)


### Neighborhood Condidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our are of interest which is approximately 12x12 kilometers centered around Kuala Lumpur city center.

Let's first find the latitude & longitude of Kuala Lumpur city center, using, specific, well known address and GeoPy geocoding.

In [4]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'KLCC'

geolocator = Nominatim(user_agent="kl_explorer")
location = geolocator.geocode(address)
kl_latitude = location.latitude
kl_longitude = location.longitude
kl_center = [kl_latitude, kl_longitude]
print('Coordinate of {}: {}'.format(address, kl_center))

Coordinate of KLCC: [3.1593058, 101.7132025]


Now let's create a grid of area candidates, equaly spaced, centered around city and within ~6km from KLCC. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhoods centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate systems (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [5]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def latlon_to_xy(lat, lon):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=48, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_latlon(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=48, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[1], lonlat[0]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Kuala Lumpur center latitude={}, longitude={}'.format(kl_center[0], kl_center[1]))
x, y = latlon_to_xy(kl_center[0], kl_center[1])
print('Kuala Lumpur center UTM X={}, Y={}'.format(x, y))
la, lo = xy_to_latlon(x, y)
print('Kuala Lumpur center latitude={}, longitude={}'.format(la, lo))

Coordinate transformation check
-------------------------------
Kuala Lumpur center latitude=3.1593058, longitude=101.7132025
Kuala Lumpur center UTM X=134613.2987436622, Y=349779.8427554232
Kuala Lumpur center latitude=3.1593057999999994, longitude=101.7132025


Let's create a **hexagonal grid of cells**: we offset every other way and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [6]:
kl_center_x, kl_center_y = latlon_to_xy(kl_center[0], kl_center[1]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = kl_center_x - 6000
x_step = 600
y_min = kl_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0 # 300 for alternate rows
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(kl_center_x, kl_center_y, x, y)
        if (distance_from_center <= 6001):
            lat, lon = xy_to_latlon(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')
#latitudes

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [7]:
#!pip install folium

import folium

In [8]:
map_kl = folium.Map(location=kl_center, zoom_start=13)
folium.Marker(kl_center, popup='KLCC').add_to(map_kl)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_kl)
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_kl)
    #folium.Marker([lat, lon]).add_to(map_kl)
#folium.Circle([latitudes[0], longitudes[0]], radius=300, color='blue', fill=False).add_to(map_kl)
#folium.Circle([latitudes[25], longitudes[25]], radius=300, color='blue', fill=False).add_to(map_kl)
map_kl

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from KLCC.

Let's now use GeoPy reverse geocoding to get approximate addresses of those locations.

In [9]:
def get_address(latitude, longitude):
    try:
        geolocator = Nominatim(user_agent="kl_explorer")
        latlon = str(latitude) + ", " + str(longitude)
        location = geolocator.reverse(latlon)
        return location
    except:
        return None

addr = get_address(kl_center[0], kl_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(kl_center[0], kl_center[1], addr))

Reverse geocoding check
-----------------------
Address of [3.1593058, 101.7132025] is: Avenue K, Jalan Ampang, Kampung Baru, Kuala Lumpur, 50400, Malaysia


In [10]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = str(address).replace(', Malaysia', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [11]:
addresses[150:170]

['Jalan Dagang 6/8, Taman Putra Sulaiman, Majlis Perbandaran Ampang Jaya, Selangor, 68000',
 'Jalan Dagang 1/4, Taman Dagang, Majlis Perbandaran Ampang Jaya, Selangor, 68000',
 'Jalan Pinang, Taman Kosas, Majlis Perbandaran Ampang Jaya, Selangor, 68000',
 'Lorong Dungun, Bukit Damansara, Kuala Lumpur, 50480',
 'Jalan Lembah Ledang, Kuala Lumpur, 50480',
 'Jalan Ledang, Taman Duta, Kuala Lumpur, 50480',
 'Lebuhraya Sultan Iskandar, Kuala Lumpur, 50580',
 'Jalan Berjasa, Chow Kit, Kuala Lumpur, 50580',
 'Asia School of Business, Jalan Sultan Salahuddin, Chow Kit, Kuala Lumpur, 50580',
 'Madras Store, Jalan Bunus 6, Chow Kit, Kuala Lumpur, 50100',
 'KL Forest Eco Park, Changkat Raja Chulan, Bukit Bintang, Kuala Lumpur, 50200',
 'Gravy Baby, Jalan Cangkat Perak, Bukit Bintang, Kuala Lumpur, 50250',
 'KLCC Convention Center, KLCC-Bukit Bintang Pedestrian Walkway, Bukit Bintang, Kuala Lumpur, 50088',
 'Naza Tower, 10, Persiaran KLCC, Bukit Bintang, Kuala Lumpur, 50088',
 'Jalan Taman U-Thant

Looking good. Let's now place all this into a Pandas dataframe.

In [12]:
import pandas as pd
df_locations = pd.DataFrame({
    'Address': addresses,
    'Latitude': latitudes,
    'Longitude': longitudes,
    'X': xs,
    'Y': ys,
    'Distance from center': distance_from_center
})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,Bandar Malaysia - High Speed Rail station to S...,3.107630,101.697191,132813.298744,344064.075090,8286.73639
1,"Razak Mansion, Kuala Lumpur, 51020",3.107647,101.702582,133413.298744,344064.075090,8286.73639
2,"Razak Mansion, Kuala Lumpur, 51020",3.107664,101.707973,134013.298744,344064.075090,8286.73639
3,"Bandar Sri Permaisuri, Cheras, Kuala Lumpur, 5...",3.107681,101.713364,134613.298744,344064.075090,8286.73639
4,"Jalan Siakep, Flat Sri Melaka (block 4E), Cher...",3.107697,101.718755,135213.298744,344064.075090,8286.73639
5,"Bandar Sri Permaisuri, Cheras, Kuala Lumpur, 5...",3.107714,101.724146,135813.298744,344064.075090,8286.73639
6,"Jalan Midah 1, Taman Midah, Cheras, Kuala Lump...",3.107731,101.729537,136413.298744,344064.075090,8286.73639
7,"Jalan Desa Seputeh 1, Seputeh Point, Seputeh, ...",3.112298,101.689090,131913.298744,344583.690333,8286.73639
8,"Seputeh, Kuala Lumpur, 50460",3.112315,101.694481,132513.298744,344583.690333,8286.73639
9,"Jalan Lapangan Terbang Lama, Razak Mansion, Ta...",3.112332,101.699872,133113.298744,344583.690333,8286.73639


... and let's now save/persist this data into local file.

In [13]:
df_locations.to_pickle('./locations.pkl')

### Foursquare ###

Now that we have our location candidates, let's use Foursquare API to get into on restaurants in each neighborhood.

We're intereseted in venues in 'drugstore' category. So we will include in out list only venues that have 'drugstore' in category name.

Foursquare credentials are defined below.

In [14]:
foursquare_client_id = 'VOIQ1IJZOVLYTNTKYMPWUYSCRUZPU5MR2QA2TJUXFR0MDX1Y' # your Foursquare ID
foursquare_client_secret = 'L5VJ03YG252BR5WOLJP1MQSSVSNG3XQ2HRFOMAVJ2TOPXWU1' # your Foursquare Secret

In [15]:
import requests

# Category IDs corresponding to Japanese restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

japanese_restaurant_categories = ['55a59bace4b013909087cb0c','55a59bace4b013909087cb30','55a59bace4b013909087cb21',
                                 '55a59bace4b013909087cb06','55a59bace4b013909087cb1b','55a59bace4b013909087cb1e',
                                 '55a59bace4b013909087cb18','55a59bace4b013909087cb24','55a59bace4b013909087cb15',
                                 '55a59bace4b013909087cb27','55a59bace4b013909087cb12','4bf58dd8d48988d1d2941735',
                                 '55a59bace4b013909087cb2d','55a59a31e4b013909087cb00','55a59af1e4b013909087cb03',
                                 '55a59bace4b013909087cb2a','55a59bace4b013909087cb0f','55a59bace4b013909087cb33',
                                 '55a59bace4b013909087cb09','55a59bace4b013909087cb36']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'ryori', 'shokuto']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Malaysia', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]
    except:
        venues = []
    return venues

In [16]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found Japanese restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    japanese_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_japanese = is_restaurant(venue_categories, specific_filter=japanese_restaurant_categories)
            if is_res:
                x, y = latlon_to_xy(venue_latlon[0], venue_latlon[1])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_japanese, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_japanese:
                    japanese_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, japanese_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
japanese_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('japanese_restaurants_350.pkl', 'rb') as f:
        japanese_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, japanese_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('japanese_restaurants_350.pkl', 'wb') as f:
        pickle.dump(japanese_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [17]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Japanese restaurants:', len(japanese_restaurants))
print('Percentage of Japanese restaurants: {:.2f}%'.format(len(japanese_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())
len(restaurants)

Total number of restaurants: 3076
Total number of Japanese restaurants: 41
Percentage of Japanese restaurants: 1.33%
Average number of restaurants in neighborhood: 7.31043956043956


3076

In [18]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('524bb499498e8db87c662c58', '斋饭', 3.1072354316711426, 101.69904327392578, 'Malaysia', 210, False, 133019.33019345696, 344019.756126759)
('4e773dbb483b0cf5ed865bf3', '(King kong) ahmed maideen sg. Besi', 3.1091277365733867, 101.70274582567103, 'Flat razak mansion (Jln sungai besi), 57100 Kuala Lumpur, Kuala Lumpur', 165, False, 133432.06781630078, 344227.9783441066)
('4ced2a1b4cd4236a4c70a663', 'Roti Petak Razak Mension', 3.1090862687223955, 101.7026679912784, '57100 Kuala Lumpur, Federal Territory of Kuala Lum', 160, False, 133423.39076133748, 344223.4142278337)
('4ef00f270e01e1fde207f569', 'Sushi King Favorites', 3.10611840356321, 101.70024959451372, 'Sougate (Jalan Sg Besi), Kuala Lumpur, Kuala Lumpur', 310, False, 133153.20385912096, 343895.66242862976)
('4dff191ac65b10468961a767', 'Sg. Besi Corner Mamak Stall', 3.1090220452093544, 101.70634256216775, 'Jalan Besi Batu 3 1/2, 57100 Kuala Lumpur, Federal Territory of Kuala Lum', 236, Fa

In [19]:
print('List of Japanese restaurants')
print('---------------------------')
for r in list(japanese_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(japanese_restaurants))

List of Japanese restaurants
---------------------------
('5d1d51e3e7e5fc002381c8c5', 'IPPUDO Gardens Mall', 3.11984, 101.67845, 'Lot-T218 3rd Floor The Gardens Mall Lingkaran Syed Putra, The Gardens Mall, 59200 Kuala Lumpur, Kuala Lumpur', 202, True, 130731.73043860408, 345422.51535982767)
('4ff17e98e4b0e65c8e1687bb', 'Shushi-zen', 3.1187941455804844, 101.67854769602702, 'Malaysia', 319, True, 130742.23814014671, 345306.68457305647)
('58537573ce593d5304d38a7a', 'Sushi King', 3.1268973379715748, 101.72448199008879, 'Sunway Velocity (Lot 4-09, 4th Floor), 55100 Kuala Lumpur, Kuala Lumpur', 63, True, 135857.32828414644, 346187.79374844977)
('58523995040d5373852e2882', 'Sushi Zanmai (寿司三味)', 3.127788775997752, 101.72428812137153, 'Sunway Velocity', 146, True, 135836.0603522791, 346286.55611480115)
('5528b073498ea4c3e3843881', 'Sushi Mentai 壽司明太', 3.128689, 101.751241, '9-G, Jalan Perubatan 2, Pandan Indah, Pandan Indah, Kuala Lumpur', 236, True, 138835.9760795721, 346376.89217853476)
('59

In [20]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: Ishq Restaurant, Museum Restaurant, Islamic Art Museum Malaysia
Restaurants around location 102: Kedai Makan City Parking, Heritage Station Restaurant, Rail Cafe, Kedai Mamak KTM, Ghandi Vegetarian Restaurant
Restaurants around location 103: Bawah Pokok Indian Rice, Restaurant Pik Wah (碧華樓酒家), 阿順哥鮑魚盤菜 Chef Soon Koh Restaurant Sdn Bhd, Ali, Muthu & Ah Hock, Wildflowers, Purple Cane 紫藤茶原, Back Alley Prawn Mee, 玉壶轩茶楼
Restaurants around location 104: Sarang Cookery, Warong Bomba Jln Hang Tuah, Vistana Corner, meeexx, 河清园源记茶餐室
Restaurants around location 105: Restoran Yap Hup Kee, Wong Mei Kee (王美记) Restaurant, Restoran Sin Hiap Kee, 福成水饺大王, 悦心軒素食館, Purple Cane, Ayam Penyet Express, The Zun Kitchenette 尊厨
Restaurants around location 106: Restaurant Onn Kee Claypot Seafood Porridge, 好想吃 Hxc Grand Ballroom, Restoran O.K. Kari Kepala Ikan (O.K.巴东咖哩鱼头餐厅海鲜粥家), Restoran New Ocean, Restoran Xin Dong Ting (新洞庭骨

Let's now see all the collected restaurants in our area of interest on map, and let's also show Japanese restaurants in different color.

In [24]:
map_kl = folium.Map(location=kl_center, zoom_start=13)
folium.Marker(kl_center, popup='KLCC').add_to(map_kl)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_japanese = res[6]
    color = 'red' if is_japanese else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_kl)
map_kl

Looking good. So now we have all the restaurants in area within few kilometers from KLCC, and we know which ones are Japanese restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Japanese restaurant!